# Data Science Final Project

In [1]:
# !pip install nltk
# !pip install keras
# !pip install tensorflow
from matplotlib import pyplot
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from datetime import datetime
import time
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import gc
from dateutil import parser
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


plt.rcParams.update(plt.rcParamsDefault)
%matplotlib inline
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tdepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


https://www.kaggle.com/datasets/kazanova/sentiment140

https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/

In [2]:
df = pd.read_csv('twitterdata.csv', encoding = "ISO-8859-1")
print('Loaded file.')
df = pd.DataFrame(np.vstack([df.columns, df])) # Moves column names into row 1
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'tweet'] # Renames columns
df.replace({'sentiment': {4: 1}}, inplace=True) # Replaces all '4's with '1's in column 'sentiment'
df.replace({'sentiment': {0: -1}}, inplace=True) # Replaces all '0's with '-1's in column 'sentiment'
data_types_dict = {
    'sentiment': int,
    'id': float
}
print('Fixed sentiment values.')
# df['unix_time'] = pd.to_datetime(df['date'], format='%a %b %d %H:%M:%S %Z %Y', errors='coerce').astype(int) / 10**9
# df.dropna(subset=['unix_time'], inplace=True)

# df['unix_time'] = df['date'].apply(lambda x: parser.parse(x, tzinfos={"PDT": -7*3600}).timestamp())
# print('Added unix column.')

df = df.astype(data_types_dict)
df.dtypes

Loaded file.
Fixed sentiment values.


sentiment      int32
id           float64
date          object
query         object
user          object
tweet         object
dtype: object

In [3]:
# df = pd.read_csv('twitterdata_unix.csv', encoding = "ISO-8859-1")
# df = df[['sentiment', 'tweet', 'unix_time']]
df = df.sample(frac=1).reset_index(drop=True) # shuffles rows so its not all 1 and then -1
df = df[['sentiment', 'tweet']]
df.head()

,sentiment,tweet
0,1,@blcarol007 WAIT! You forgot to send them up ...
1,1,http://tinyurl.com/ry9wap Good day! I like tra...
2,1,@firemanfrig623 Enjoy her! They grow up too fa...
3,-1,Power cut
4,-1,I just bricked my iPhone. Wheee!


In [4]:
s = df.stack().value_counts()
print(s)

1                                                               800000
-1                                                              799999
isPlayer Has Died! Sorry                                           210
good morning                                                       118
headache                                                           115
                                                                 ...  
tired ..  what a nite.! oO hun? was great (:.. luv u!!xxx            1
i love my nephew ...  have started chapter 19 of twilight =]         1
Fuck poles                                                           1
ICONic3 ...... why are you NOT following me ?                        1
@crdbl I'M NOT COCKY.                                                1
Length: 1581469, dtype: int64


In [5]:
# for i in range(10):
#     print(df['tweet'][i])
content = df['tweet'][:1000].values.tolist()
content = [x.strip() for x in content] # Deletes white space before and after
content[:20]


['@blcarol007 WAIT!  You forgot to send them up to MN first!  Silly  #YATrivia',
 'http://tinyurl.com/ry9wap Good day! I like traveling.  Please tell me more about yourself, I am interested... I can send you my pics,  ...',
 '@firemanfrig623 Enjoy her! They grow up too fast  Love you!',
 'Power cut',
 'I just bricked my iPhone. Wheee!',
 'Getting mixed signals from him....idk what to do....',
 '@prettyyinpink my rink  i dont know how im going to afford ice time now the prices are insane',
 "hi rey!  hope you're good.",
 '@peter_avery LMAO! Is that Dawson? Aw! No more Beaker huh?  LMAO',
 'Winston went potty outside all by himself! such a proud momma! Now off to get my tooth drilled by my uncle...',
 '@emilysiren @michellesmiles I know! He is growing up!',
 'hey twitter!! i forgot about you sorry',
 "So back from the spa/sister trip. It was awesome. i'm so mellow. Even Monday doesn't scare me.",
 "finished weekend errands so it's time to take a long ride on the motorcycle",
 "I need exp

## stop words #1

In [6]:
def full_remove(x, removal_list):
    for w in removal_list:
        x = x.replace(w, ' ')
    return x

## Remove digits
digits = [str(x) for x in range(10)]
remove_digits = [full_remove(x, digits) for x in content]

## Remove punctuation
remove_punc = [full_remove(x, list(string.punctuation)) for x in remove_digits]

## Make everything lower-case and remove any white space
sents_lower = [x.lower() for x in remove_punc]
sents_lower = [x.strip() for x in sents_lower]

## Remove stop words
from nltk.corpus import stopwords
stops = stopwords.words("English")
def removeStopWords(stopWords, txt):
    newtxt = ' '.join([word for word in txt.split() if word not in stopWords])
    return newtxt
content1 = [removeStopWords(stops,x) for x in sents_lower]
content1[:20]

['blcarol wait forgot send mn first silly yatrivia',
 'http tinyurl com ry wap good day like traveling please tell interested send pics',
 'firemanfrig enjoy grow fast love',
 'power cut',
 'bricked iphone wheee',
 'getting mixed signals idk',
 'prettyyinpink rink dont know im going afford ice time prices insane',
 'hi rey hope good',
 'peter avery lmao dawson aw beaker huh lmao',
 'winston went potty outside proud momma get tooth drilled uncle',
 'emilysiren michellesmiles know growing',
 'hey twitter forgot sorry',
 'back spa sister trip awesome mellow even monday scare',
 'finished weekend errands time take long ride motorcycle',
 'need exposure sales hurt either',
 'ayudevina little sad fine watching hills hahaha',
 'oceanup ugh gross',
 'missmaria need jobs',
 'mgpotter cute',
 'robins back sign blue tits days']

## stops we defined instead

In [7]:
stop_set = ['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']
content2 = [removeStopWords(stop_set,x) for x in sents_lower]
content2[:20]

['blcarol wait you forgot send them up mn first silly yatrivia',
 'http tinyurl com ry wap good day like traveling please tell me more about yourself am interested can send you my pics',
 'firemanfrig enjoy her grow up too fast love you',
 'power cut',
 'just bricked my iphone wheee',
 'getting mixed signals him idk what do',
 'prettyyinpink my rink dont know how im going afford ice time now prices are insane',
 'hi rey hope you re good',
 'peter avery lmao is that dawson aw no more beaker huh lmao',
 'winston went potty outside all by himself such proud momma now off get my tooth drilled by my uncle',
 'emilysiren michellesmiles know is growing up',
 'hey twitter forgot about you sorry',
 'so back spa sister trip was awesome m so mellow even monday doesn t scare me',
 'finished weekend errands so s time take long ride on motorcycle',
 'need exposure and few sales wouldn t hurt either',
 'ayudevina little sad but fine m watching hills hahaha and you',
 'oceanup ugh gross',
 'missmaria 

## stemming

In [8]:
import nltk
def stemporter(words):
    porter = nltk.PorterStemmer()
    new_words = [porter.stem(w) for w in words]
    return new_words
    
def stemlancaster(words):
    porter = nltk.LancasterStemmer()
    new_words = [porter.stem(w) for w in words]
    return new_words    

porter = [stemporter(x.split()) for x in content2]
porter = [" ".join(i) for i in porter]
porter[0:10]

['blcarol wait you forgot send them up mn first silli yatrivia',
 'http tinyurl com ry wap good day like travel pleas tell me more about yourself am interest can send you my pic',
 'firemanfrig enjoy her grow up too fast love you',
 'power cut',
 'just brick my iphon wheee',
 'get mix signal him idk what do',
 'prettyyinpink my rink dont know how im go afford ice time now price are insan',
 'hi rey hope you re good',
 'peter averi lmao is that dawson aw no more beaker huh lmao',
 'winston went potti outsid all by himself such proud momma now off get my tooth drill by my uncl']

## vectorizing

In [9]:
vectorizer = CountVectorizer(analyzer = "word", 
                             preprocessor = None, 
                             stop_words =  'english', 
                             max_features = 6000, ngram_range=(1,5))
data_features = vectorizer.fit_transform(sents_processed)
tfidf_transformer = TfidfTransformer()
data_features_tfidf = tfidf_transformer.fit_transform(data_features)
data_mat = data_features_tfidf.toarray()

NameError: name 'sents_processed' is not defined